In [1]:
#Mcallen
#San Antonio
#Edinburg
#Austin
#Dallas
#San Marcos
#Corpus Christi
#Weslaco
#Boerne
#Houston

### Imports

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import urllib.parse

### HTTP Request

In [2]:
#store website in a variable
website = 'https://www.trulia.com/TX/Austin/'

In [3]:
#set up selenium webdriver
driver = webdriver.Chrome()

#get the webpage using selenium

try:
    # Get the webpage using Selenium
    driver.get(website)
    time.sleep(5)  # Give the page some time to load

    # Scroll down to load more content
    for _ in range(20):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Adjust the delay based on the page loading speed

    # Get the updated page source after scrolling
    updated_html = driver.page_source

    # Create a BeautifulSoup object using the updated HTML
    soup = BeautifulSoup(updated_html, 'html.parser')

    # Extract the desired results with data-testid
    result_container = soup.find_all(attrs={'data-testid': True})


    # Retrieve anchor tags into a list and establish base URL
    url_part_1 = 'https://www.trulia.com'
    url_part_2 = []

    for item in result_container:
        for link in item.find_all('div', {'data-testid': 'property-card-details'}):
            url_part_2.append(link.find('a').get('href'))

finally:
    driver.quit()  # Close the browser window when done

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event): operation timed out


In [4]:
# Remove spaces and duplicates from list
url_only = [x.strip(' ') for x in url_part_2]
url_only = list(dict.fromkeys(url_only))
len(url_only)

40

In [5]:
# Concatenate URL
url_joined = []
for link_2 in url_only:
    url_joined.append(urllib.parse.urljoin(url_part_1,link_2))

In [6]:
url_joined

['https://www.trulia.com/home/1203-sarabeth-way-austin-tx-78721-331620941',
 'https://www.trulia.com/home/1129-walton-ln-2-austin-tx-78721-345190532',
 'https://www.trulia.com/home/1129-walton-ln-1-austin-tx-78721-345190268',
 'https://www.trulia.com/home/1406-windsor-rd-203-austin-tx-78703-29329061',
 'https://www.trulia.com/home/200-congress-ave-32se-austin-tx-78701-121655288',
 'https://www.trulia.com/home/4324-Airport-Blvd-Austin-TX-78722-29399669',
 'https://www.trulia.com/home/17800-dufour-dr-austin-tx-78738-331627452',
 'https://www.trulia.com/home/1708-hether-st-austin-tx-78704-29322269',
 'https://www.trulia.com/home/12608-padua-dr-austin-tx-78739-119621415',
 'https://www.trulia.com/home/7811-tisdale-dr-austin-tx-78757-29420720',
 'https://www.trulia.com/home/15101-parakeet-st-austin-tx-78734-144953782',
 'https://www.trulia.com/home/12330-shropshire-blvd-austin-tx-78753-29437011',
 'https://www.trulia.com/home/5325-western-hills-dr-austin-tx-78731-29345747',
 'https://www.tr

### Get data from links

In [7]:
# address
# bedrooms
# bathrooms
# sqft
# year built
# parking
# price

In [8]:
first_link = url_joined[0]
print(first_link)

https://www.trulia.com/home/1203-sarabeth-way-austin-tx-78721-331620941


### Get request and Soup Object

In [9]:
response = requests.get(first_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [10]:
soup = BeautifulSoup(response.content, 'html.parser')

In [1]:
# Commenting this step out to avoid copyright
#soup

#### Put all of the attributes together and loop through results from page 1


In [12]:
address = []
bedrooms = []
bathrooms = []
area = []
year_built = []
parking = []
price = []

# Loop through all joined links
for link in url_joined:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})
    
    # Create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Address
    try:
        address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
    except:
        address.append('')
    # Bedrooms  
    try:
        bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
    except:
        bedrooms.append('')
        
    # Bathrooms
    try:
        bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
    except:
        bathrooms.append('')
        
    # Square Feet
    try:
        area.append(soup.find('li', {'data-testid':'floor'}).get_text())
    except:
        area.append('')
    
    # Year Built
    try:
        year_built.append(soup.find('div',string='Year Built').findNext('span').get_text())
        year_built = [x.strip('Year Built: ') for x in year_built] 
    except:
        year_built.append('')
    
    # Parking
    try:
        parking.append(soup.find(string='Parking').findNext('div').get_text())
    except:
        parking.append('')
    
    # Price
    try:
        price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
    except:
        price.append('') 
    
    # Create a dictionary with results
    output = {'Address': address, 'Bathrooms': bathrooms, 'Area':area, 'Year Built':year_built, 
              'Parking':parking, 'Price':price}
    
output

{'Address': ['1203 Sarabeth Way',
  '1129 Walton Ln #2',
  '1129 Walton Ln #1',
  '1406 Windsor Rd #203',
  '200 Congress Ave #32SE',
  '4324 Airport Blvd',
  '17800 Dufour Dr',
  '1708 Hether St',
  '12608 Padua Dr',
  '7811 Tisdale Dr',
  '15101 Parakeet St',
  '12330 Shropshire Blvd',
  '5325 Western Hills Dr',
  '3807 Rockledge Dr',
  '2301 Lawnmont Ave #16',
  '907 Speer Ln',
  '2512 Wheless Ln #6',
  '1127 Walton Ln #2',
  '1106 Blair Way',
  '1127 Walton Ln #1',
  '2806 Firecrest Dr',
  '7233 Menchaca Rd #30',
  '10317 English Oak Dr',
  '4001 Verano Dr',
  '1900 Barton Springs Rd #5009',
  '201 Challis Ct',
  '5816 Abilene Trl',
  'Nova Plan in Skyridge by MileStone Community Builders',
  '6708 Menchaca Rd #1',
  '7301 Fire Cv',
  '1615 E  7th St #205',
  '300 Bowie St #3803',
  '632 Santaluz Path',
  '2401 E  6th St   #6089',
  '112 Burros Tail Cv',
  '2414 E  8th St #2',
  '1748 Ohlen Rd #43',
  '1611 Airedale Rd #1502',
  '6400 Amberly Pl',
  '4704 Blue Meadow Dr'],
 'Bathro

In [13]:
# Create dataframe
df = pd.DataFrame(output)
df

,Address,Bathrooms,Area,Year Built,Parking,Price
0,1203 Sarabeth Way,3 Baths,"1,657 sqft",2021,1 Car Garage,"$720,000"
1,1129 Walton Ln #2,3 Baths,"1,094 sqft",2024,No Info,"$685,000"
2,1129 Walton Ln #1,3 Baths,"1,814 sqft",2024,Garage,"$975,000"
3,1406 Windsor Rd #203,1 Bath,856 sqft,1982,No Info,"$410,000"
4,200 Congress Ave #32SE,3 Baths,"2,082 sqft",2008,2 Car Garage,"$3,395,000"
5,4324 Airport Blvd,,,,No Info,"$595,000"
6,17800 Dufour Dr,4 Baths,"3,403 sqft (on 0.30 acres)",2021,3 Car Garage,"$995,000"
7,1708 Hether St,5 Baths,"3,615 sqft",2016,Garage,"$3,675,000"
8,12608 Padua Dr,5 Baths,"4,159 sqft",2014,3 Car Garage,"$1,500,000"
9,7811 Tisdale Dr,2 Baths,"1,071 sqft",1958,Attached Garage,"$599,555"


In [14]:
# Add location to dataframe
df['Location'] = 'Austin'
df

,Address,Bathrooms,Area,Year Built,Parking,Price,Location
0,1203 Sarabeth Way,3 Baths,"1,657 sqft",2021,1 Car Garage,"$720,000",Austin
1,1129 Walton Ln #2,3 Baths,"1,094 sqft",2024,No Info,"$685,000",Austin
2,1129 Walton Ln #1,3 Baths,"1,814 sqft",2024,Garage,"$975,000",Austin
3,1406 Windsor Rd #203,1 Bath,856 sqft,1982,No Info,"$410,000",Austin
4,200 Congress Ave #32SE,3 Baths,"2,082 sqft",2008,2 Car Garage,"$3,395,000",Austin
5,4324 Airport Blvd,,,,No Info,"$595,000",Austin
6,17800 Dufour Dr,4 Baths,"3,403 sqft (on 0.30 acres)",2021,3 Car Garage,"$995,000",Austin
7,1708 Hether St,5 Baths,"3,615 sqft",2016,Garage,"$3,675,000",Austin
8,12608 Padua Dr,5 Baths,"4,159 sqft",2014,3 Car Garage,"$1,500,000",Austin
9,7811 Tisdale Dr,2 Baths,"1,071 sqft",1958,Attached Garage,"$599,555",Austin


In [15]:
df.to_excel('dataframe_austin.xlsx', index=False)